In [13]:
# Importando bibliotecas

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
from random import randint

In [14]:
import bs4

print(pd.__version__)
print(np.__version__)
print(bs4.__version__)
print(requests.__version__)

1.4.1
1.22.4
4.10.0
2.26.0


In [15]:
# Função que altera multiplos caracteres de uma lista

def special_replace(lista, string, special_c = ""):
    for i in lista:
        string = string.replace(i, special_c)
    return string

In [16]:
# Requisita e retorna o codigo fonte da pagina

def get_data(url):
    
    html_text = ''
    while html_text == '':
        try:
            html_text = requests.get(url).text
            soup = BeautifulSoup(html_text, 'lxml')
        except:
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            time.sleep(5)
            print("Was a nice sleep, now let me continue...")
    return soup

In [17]:
# Retorna uma string contendo apenas os seus caracteres numericos

def get_numeric(string):
    numeric_string = ''
    for char in string:
        if char.isnumeric():
            numeric_string += char
    return numeric_string

In [18]:
# Retorna a quantidade de espisodios do anime e sua data de lançamento em uma lista

def get_info(soup):
    
    info = soup.find('div', class_ = 'information di-ib mt4').text.replace(",", ".")
    indesejados = [' ', 'TV(', 'OVA(', 'Movie(', 'ONA(', 'Special(', ')', 'members', 'eps', 'Music(', '?']
    info = special_replace(indesejados, info)
    info = info.split('\n')[1:-2]
    info[1] = get_numeric(info[1].split('-')[0])
    return info

In [19]:
# Retorna uma lista com todos os animes no formato de dicionario

def get_animes(soup):
    score = 9
    can_score = True
    obras = []
    animes = soup.find_all('tr', class_ = 'ranking-list')
    for anime in animes:
        nome = anime.find('h3', class_ = 'hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3').text
        info = get_info(anime)
        try:
            eps = int(info[0])
        except:
            eps = np.nan
        try:
            lancamento = int(info[1])
        except:
            lancamento = np.nan
        while True:
            try:
                nota = float(anime.find('span', class_ = f'text on score-label score-{score}').text)
                break
            except:
                score -= 1
                if score < 0:
                    can_score = False
                    break
        if not can_score:
            break
        mais_info = anime.find('h3', class_ = "hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3").a['href']
        
        obra = {
            'Nome': nome,
            'Quantidade de Episódios': eps,
            'Data de lançamento': lancamento,
            'Nota': nota,
            'Mais informações': mais_info
        }
        
        obras.append(obra)
    return obras

In [20]:
# Salva os animes em um csv

def output(lista_animes):
    df_animes = pd.concat(pd.DataFrame(a) for a in lista_animes)
    df_animes.to_csv('animes_mal.csv', index=False)
    print('Fim. Animes salvos no csv')
    print(df_animes.head())
    return

In [21]:
# Main

if __name__ == '__main__':
    
    resultados = []
    limit = 13800
    tempo = randint(1, 5)
    for i in range(0, limit, 50):    
        url = f"https://myanimelist.net/topanime.php?limit={i}"
        soup = get_data(url)
        resultados.append(get_animes(soup))
        time.sleep(tempo)
    output(resultados)

Fim. Animes salvos no csv
                                                Nome  Quantidade de Episódios  \
0  Shingeki no Kyojin: The Final Season - Kankets...                      2.0   
1                   Fullmetal Alchemist: Brotherhood                     64.0   
2                          Bleach: Sennen Kessen-hen                     13.0   
3                                        Steins;Gate                     24.0   
4                                           Gintama°                     51.0   

   Data de lançamento  Nota                                   Mais informações  
0              2023.0  9.17  https://myanimelist.net/anime/51535/Shingeki_n...  
1              2009.0  9.11  https://myanimelist.net/anime/5114/Fullmetal_A...  
2              2022.0  9.09  https://myanimelist.net/anime/41467/Bleach__Se...  
3              2011.0  9.08     https://myanimelist.net/anime/9253/Steins_Gate  
4              2015.0  9.07       https://myanimelist.net/anime/28977/Gintama°  
